In [130]:
import pandas as pd
import numpy as np

class ModelContributionAmounts:
    def __init__(self, minimum_eligible_amount, apy):
        """
        Initialize the contribution amounts model.

        Parameters:
        - minimum_eligible_amount (float): Minimum treatment cost to be eligible for the program.
        - apy (float): Annual Percentage Yield for interest calculations.
        """
        self.minimum_eligible_amount = minimum_eligible_amount
        self.apy = apy

    def process_transactions(self, transaction_df, conversion_rate, monthly_payment_rate):
        """
        Process transactions to determine contribution program eligibility and generate schedules.

        Parameters:
        - transaction_df (pd.DataFrame): Input transaction DataFrame with 'Customer ID', 'Period', and 'Revenue'.
        - conversion_rate (float): Probability of a transaction being keen for conversion.
        - monthly_payment_rate (float): Percentage of the treatment price as the monthly contribution.

        Returns:
        - pd.DataFrame: Updated transaction DataFrame with a 'Convert' column.
        - pd.DataFrame: Resulting contribution schedule DataFrame.
        """
        
        transaction_df['Period'] = pd.to_datetime(transaction_df['Period'])
        
        # Initialize columns
        transaction_df['Convert'] = False

        # Determine the earliest possible date
        min_period = transaction_df['Period'].min()

        # Initialize result DataFrame
        result_data = []

        for idx, row in transaction_df.iterrows():
            customer_id = row['Customer ID']
            treatment_date = row['Period']
            treatment_price = row['Revenue']

            # Skip if treatment price is below minimum eligible amount
            if treatment_price < self.minimum_eligible_amount:
                continue

            # Apply conversion rate
            if np.random.rand() > conversion_rate:
                continue

            # Calculate monthly payment
            monthly_payment = treatment_price * monthly_payment_rate
            monthly_apy = (1 + self.apy) ** (1 / 12) - 1
            nper = 0
            accrued_interest = 0
            total_future_value = 0
            interest_list = []
            
            # random select integer from 1 to 9 for nper_max
            nper_max = np.random.randint(3, 10) 

            # Calculate nper iteratively to include accrued interest
            while total_future_value < treatment_price and nper <= nper_max:
                

                           
                
                nper += 1
                
                total_future_value += monthly_payment
                interest = total_future_value * monthly_apy
                interest_list.append(interest)
                
                
                # future_value = monthly_payment * (1 + self.apy / 12) ** (nper - 1)
                
                accrued_interest += interest
                
                
                if treatment_price - (total_future_value + accrued_interest) < 0:
                    
                    nper -= 1
                    total_future_value -= monthly_payment
                    accrued_interest -= interest_list[-1]
                    interest_list.pop()
                
                
                    break
                

            
            
            

            

            # Check if nper satisfies the time delta condition
            time_delta = (treatment_date - min_period).days
            if time_delta < 30 * nper:
                continue
            
            print(f"customer_id: {customer_id}")
            print(f"total_future_value: {total_future_value}")
            print(f"treatment_price: {treatment_price}")
            print(f"accrued_interest: {accrued_interest}")
            print(f"Monthly Payment: {monthly_payment}")
            
            print(f"total deposits: {(monthly_payment * nper) + accrued_interest}")
            
            print(f"nper: {nper}")

            # Eligible transaction, mark as Convert
            transaction_df.at[idx, 'Convert'] = True

            # Generate contribution schedule
            first_deposit_date = treatment_date - pd.Timedelta(days=30 * (nper))
            for i in range(nper):
                deposit_date = first_deposit_date + pd.Timedelta(days=30 * i)
                result_data.append({
                    'Period': deposit_date,
                    'Customer ID': customer_id,
                    'Revenue': monthly_payment,
                    'Expense': 0,
                    'Type': 'Deposit'
                })
                
            for i in interest_list:
                deposit_date = first_deposit_date + pd.Timedelta(days=30 * i)
                result_data.append({
                    'Period': deposit_date,
                    'Customer ID': customer_id,
                    'Revenue': i,
                    'Expense': 0,
                    'Type': 'Interest'
                 })
            

            # Add final payment row
            remaining_fee = treatment_price - (total_future_value + accrued_interest)
            result_data.append({
                'Period': treatment_date,
                'Customer ID': customer_id,
                'Revenue': remaining_fee,
                'Expense': 0,
                'Type': 'Final Payment'
            })
            
            print(f"remaining_fee: {remaining_fee}")
            print(f"remaining fee + total deposits: {remaining_fee + (monthly_payment * nper) + accrued_interest}")
            
            print("---")
            
            
        
        # write me 
        for idx, row in transaction_df.iterrows():
            if row['Convert']:
                result_data.append({
                    'Period': row['Period'],
                    'Customer ID': row['Customer ID'],
                    'Revenue': 0,
                    'Expense': row['Revenue'],
                    'Type': 'Cashback'
                })
        
        
        
        # Create result DataFrame
        result_df = pd.DataFrame(result_data)
        result_df['Period'] = pd.to_datetime(result_df['Period']).dt.date
        
        return transaction_df, result_df

    def validate_result(self, updated_transaction_df, result_df):
        """
        Validate that the result DataFrame correctly calculates deposits and interest.

        Parameters:
        - updated_transaction_df (pd.DataFrame): Transaction DataFrame with 'Convert' column.
        - result_df (pd.DataFrame): Contribution schedule DataFrame.

        Returns:
        - list: Validation issues, if any.
        """
        validation_issues = []

        for customer_id in updated_transaction_df.loc[updated_transaction_df['Convert'] == True, 'Customer ID']:
            # Filter result_df for this customer
            customer_payments = result_df[result_df['Customer ID'] == customer_id]

            # Separate deposit and final payment rows
            deposits = customer_payments[customer_payments['Type'] == 'Deposit']
            final_payment_row = customer_payments[customer_payments['Type'] == 'Final Payment']

            # Recalculate accrued interest
            treatment_date = final_payment_row['Period'].iloc[0]
            total_accrued_interest = 0
            for _, deposit_row in deposits.iterrows():
                deposit_date = deposit_row['Period']
                months_remaining = ((treatment_date - deposit_date).days) // 30
                future_value = deposit_row['Amount'] * (1 + self.apy / 12) ** months_remaining
                accrued_interest = future_value - deposit_row['Amount']
                total_accrued_interest += accrued_interest

            # Calculate totals
            total_deposits = deposits['Amount'].sum()
            final_payment = final_payment_row['Amount'].iloc[0]
            treatment_price = updated_transaction_df.loc[
                (updated_transaction_df['Customer ID'] == customer_id) & (updated_transaction_df['Convert'] == True), 'Revenue'
            ].iloc[0]

            # Compare with treatment price
            calculated_total = total_deposits + total_accrued_interest + final_payment
            if not np.isclose(calculated_total, treatment_price, atol=0.01):
                validation_issues.append({
                    'Customer ID': customer_id,
                    'Total Deposits': total_deposits,
                    'Accrued Interest': total_accrued_interest,
                    'Final Payment': final_payment,
                    'Treatment Price': treatment_price,
                    'Calculated Total': calculated_total
                })

        return validation_issues

In [131]:


transactions_df = pd.read_csv('forecast_df_treatment.csv')

# Initialize the model
contribution_model = ModelContributionAmounts(minimum_eligible_amount=1000, apy=0.05)

# Process transactions
updated_transactions_df, result_df = contribution_model.process_transactions(
    transactions_df,
    conversion_rate=0.5,
    monthly_payment_rate=0.2
)



customer_id: Patient 738
total_future_value: 838.4
treatment_price: 1048.0
accrued_interest: 8.53936345052695
Monthly Payment: 209.60000000000002
total deposits: 846.939363450527
nper: 4
remaining_fee: 201.0606365494731
remaining fee + total deposits: 1048.0000000000002
---
customer_id: Patient 184
total_future_value: 600.0
treatment_price: 1500.0
accrued_interest: 3.666711405283518
Monthly Payment: 300.0
total deposits: 603.6667114052835
nper: 2
remaining_fee: 896.3332885947165
remaining fee + total deposits: 1500.0
---
customer_id: Patient 567
total_future_value: 8900.800000000001
treatment_price: 11126.0
accrued_interest: 90.65740243374317
Monthly Payment: 2225.2000000000003
total deposits: 8991.457402433743
nper: 4
remaining_fee: 2134.5425975662565
remaining fee + total deposits: 11126.0
---
customer_id: Patient 311
total_future_value: 1331.2
treatment_price: 1664.0
accrued_interest: 13.55868395198172
Monthly Payment: 332.8
total deposits: 1344.7586839519818
nper: 4
remaining_fee: 

In [132]:
result_df

,Period,Customer ID,Revenue,Expense,Type
0,2025-01-03,Patient 738,209.600000,0.0,Deposit
1,2025-02-02,Patient 738,209.600000,0.0,Deposit
2,2025-03-04,Patient 738,209.600000,0.0,Deposit
3,2025-04-03,Patient 738,209.600000,0.0,Deposit
4,2025-01-28,Patient 738,0.853936,0.0,Interest
...,...,...,...,...,...
239,2025-10-17,Patient 219,0.000000,11126.0,Cashback
240,2025-10-26,Patient 323,0.000000,2211.0,Cashback
241,2025-10-30,Patient 410,0.000000,1042.0,Cashback
242,2025-12-07,Patient 42,0.000000,1025.0,Cashback


In [111]:
# filter for negative value in column amount for result_df
result_df[result_df['Amount'] < 0]

,Period,Customer ID,Amount,Type


In [112]:
result_df[result_df['Customer ID'] == 'Patient 446']

,Period,Customer ID,Amount,Type
0,2025-02-24,Patient 446,265.600000,Deposit
1,2025-03-26,Patient 446,265.600000,Deposit
2,2025-03-28,Patient 446,1.082087,Interest
3,2025-04-29,Patient 446,2.164175,Interest
4,2025-04-25,Patient 446,793.553738,Final Payment


In [113]:
result_df[result_df['Customer ID'] == 'Patient 446']['Amount'].sum()

1328.0

In [114]:
updated_transactions_df[updated_transactions_df['Customer ID'] == 'Patient 446']

,Period,Treatment,Revenue,Expense,Customer ID,Convert
457,2025-04-25,613,1328.0,200.61,Patient 446,True
497,2025-05-02,051,0.0,125.00,Patient 446,False


In [87]:
def validate_result(updated_transaction_df, result_df, apy):
    validation_issues = []

    # Group payments by Customer ID
    for customer_id in updated_transaction_df.loc[updated_transaction_df['Convert'] == True, 'Customer ID']:
        # Filter result_df for this customer
        customer_payments = result_df[result_df['Customer ID'] == customer_id]

        # Separate deposit and final payment rows
        deposits = customer_payments[customer_payments['Type'] == 'Deposit']
        final_payment_row = customer_payments[customer_payments['Type'] == 'Final Payment']

        # Recalculate accrued interest
        treatment_date = final_payment_row['Period'].iloc[0]
        total_accrued_interest = 0
        for _, deposit_row in deposits.iterrows():
            deposit_date = deposit_row['Period']
            months_remaining = ((treatment_date - deposit_date).days) // 30
            future_value = deposit_row['Amount'] * (1 + apy / 12) ** months_remaining
            accrued_interest = future_value - deposit_row['Amount']
            total_accrued_interest += accrued_interest

        # Calculate totals
        total_deposits = deposits['Amount'].sum()
        final_payment = final_payment_row['Amount'].iloc[0]
        treatment_price = updated_transaction_df.loc[
            (updated_transaction_df['Customer ID'] == customer_id) & (updated_transaction_df['Convert'] == True), 'Revenue'
        ].iloc[0]

        # Compare with treatment price
        calculated_total = total_deposits + total_accrued_interest + final_payment
        if not np.isclose(calculated_total, treatment_price, atol=0.01):
            validation_issues.append({
                'Customer ID': customer_id,
                'Total Deposits': total_deposits,
                'Accrued Interest': total_accrued_interest,
                'Final Payment': final_payment,
                'Treatment Price': treatment_price,
                'Calculated Total': calculated_total
            })

    return validation_issues


In [88]:
validate_result(updated_transactions_df, result_df, apy=0.05)

[{'Customer ID': 'Patient 45',
  'Total Deposits': 1257.6000000000001,
  'Accrued Interest': 13.154697143373824,
  'Final Payment': 301.59095482420935,
  'Treatment Price': 1572.0,
  'Calculated Total': 1572.3456519675833},
 {'Customer ID': 'Patient 22',
  'Total Deposits': 1658.0,
  'Accrued Interest': 20.84049929801472,
  'Final Payment': -20.264691699866944,
  'Treatment Price': 1658.0,
  'Calculated Total': 1658.5758075981478},
 {'Customer ID': 'Patient 462',
  'Total Deposits': 1328.0,
  'Accrued Interest': 16.692510897324325,
  'Final Payment': -16.231309154054998,
  'Treatment Price': 1328.0,
  'Calculated Total': 1328.4612017432694},
 {'Customer ID': 'Patient 239',
  'Total Deposits': 6802.400000000001,
  'Accrued Interest': 42.574048611111266,
  'Final Payment': 10162.029270561165,
  'Treatment Price': 17006.0,
  'Calculated Total': 17007.003319172276},
 {'Customer ID': 'Patient 33',
  'Total Deposits': 884.4000000000001,
  'Accrued Interest': 7.390493547453673,
  'Final Payme

## Trial with some ideas of code

In [8]:
import pandas as pd

def nper_calculation_with_schedule(treatment_price, deposit, apy):
    """
    Calculate the number of periods (nper) required to reach the treatment price
    given the monthly deposit and annual percentage yield (APY).

    Generate a schedule of contributions showing the period, deposit, accrued interest,
    and cumulative total of deposits and interest.

    Parameters:
    - treatment_price (float): The future value (FV) representing the treatment price.
    - deposit (float): The fixed monthly deposit (PMT).
    - apy (float): The annual percentage yield (as a decimal).

    Returns:
    - schedule_df (DataFrame): A schedule showing the contributions over time.
    """
    # Convert APY to monthly rate
    monthly_rate = apy / 12

    # Ensure the deposit is sufficient to accumulate interest towards the treatment price
    if deposit * monthly_rate >= treatment_price:
        raise ValueError("Monthly deposit is too small to reach the treatment price with the given APY.")

    # Initialize variables
    total_deposit = 0
    total_interest = 0
    schedule_data = []

    period = 0
    while total_deposit + total_interest < treatment_price:
        period += 1
        # Calculate interest for the current period based on accumulated deposits
        interest = total_deposit * monthly_rate
        # Update totals
        total_interest += interest
        total_deposit += deposit
        # Append data for this period
        schedule_data.append({
            "Period": period,
            "Deposit": deposit,
            "Accrued Interest": interest,
            "Total Deposits": total_deposit,
            "Cumulative Total (Deposits + Interest)": total_deposit + total_interest
        })

    # Create a DataFrame from the schedule data
    schedule_df = pd.DataFrame(schedule_data)

    return schedule_df


# Example usage
treatment_price_example = 1300  # Example treatment price
deposit_example = 100          # Example monthly deposit
apy_example = 0.12             # Example APY (5% annual)

schedule_df_example = nper_calculation_with_schedule(treatment_price_example, deposit_example, apy_example)
schedule_df_example


,Period,Deposit,Accrued Interest,Total Deposits,Cumulative Total (Deposits + Interest)
0,1,100,0.0,100,100.0
1,2,100,1.0,200,201.0
2,3,100,2.0,300,303.0
3,4,100,3.0,400,406.0
4,5,100,4.0,500,510.0
5,6,100,5.0,600,615.0
6,7,100,6.0,700,721.0
7,8,100,7.0,800,828.0
8,9,100,8.0,900,936.0
9,10,100,9.0,1000,1045.0


In [73]:
list = [1,2,3,4,5]

schedule_data = []

for i in list:
    schedule_data.append({
    "Period": i

})

In [74]:
schedule_data

[{'Period': 1}, {'Period': 2}, {'Period': 3}, {'Period': 4}, {'Period': 5}]